## ROC&BSS 检验模块
---
说明：针对三类数据（CSM106，POMMA，UMKO）进行检验，并产出相应图件。

### STEP 1.数据预处理
- CSM.py:
 处理中国气候中心的CSM106数据。
  1. 将指定目录下的所有数据读入；
  2. 基于各个月的月平均值据平；
  3. 导出NETCDF4 文件到指定目录；
  
- pomma.py:
 处理澳洲的POMMA数据数据。
  1. 将指定目录下的所有数据读入；
  2. 基于各个月的月平均值据平；
  3. 导出NETCDF4 文件到指定目录；

- UMKO.py:
 处理英国数据。
  1. 将指定目录下的所有数据读入；
  2. 基于各个月的月平均值据平；
  3. 导出NETCDF4 文件到指定目录；
  
---
### STEP 2.样本序列化
结合第一步得到的概率预报值，同时导入实际观测值，将网格上每个格点作为一个样本，
生成相应的键值对（真实值：三分类概率预报值），eg:
>y_true=[1,0,0] y_score=[0.15,0.5,0.35]

结果导出以CSV格式导出

- Serialization.py
 序列化样本操作。<br>
 1.读入观测数据和第一步得到的概率数据；<br>
 2.数据点匹配，计算格点概率；<br>
 3.序列化，以python pandas Dataframe表单保存数据；<br>
 4.结果以CSV或其他格式导出
  
---
### STEP 3.绘图
基于第二步得出的结果，可以依据需求绘制ROC曲线或BSS的网格图。
- ROC.py:
 绘制ROC曲线。
  1. 导入序列化样本文件；
  2. matplotlib 绘制图件并导出到指定目录；
  
- BSS.py:
 绘制BSS曲线。
  1. 导入序列化样本文件；
  2. matplotlib 绘制图件并导出到指定目录；

---

## 2.样本序列化

### 2.1.1 Serialize显示帮助信息

In [1]:
%run ../src/Serialize.py -h

usage: Serialize.py [-h] [-t {single,multiple}] [-n NAME] [-v] pre obs

positional arguments:
  pre                   概率预测文件路径
  obs                   观测文件路径

optional arguments:
  -h, --help            show this help message and exit
  -t {single,multiple}, --type {single,multiple}
                        文件保存方式,默认single方式
  -n NAME, --name NAME  结果文件名
  -v, --verbose         输出详细信息


1. 必填参数：Serialize.py 必须的是两个参数，由第一步导出的概率预测文件（pre）和时间上匹配的观测文件路径(obs)，这个时间匹配指的是pre数据中的每个时间（例如CSM106数据是逐月的资料），观测文件也必须有相应的时间对应上。程序中是针对每一个时间去寻找观测文件中的对应值，提取后作为完整的一个样本保存。
2. 选填参数（option）：除了通用的-h,-v外，-t,--type表示选择结果文件的存储方式，只能在［single,multiple］中二选一，顾名思义，single将所有样本数据都保存到一个文件中，而multiple则是针对每个时间都进行保存。-n，--name表示结果文件的名称。

### 2.1.2 示例：测试CSM106 1990-2015 降水资料

In [ ]:
%run ../src/Serialize.py ../tmp/data.nc ../tmp/obpre.nc -v

============parameters===========
pre ../tmp/data.nc
type single
obs ../tmp/obpre.nc
name ./tmp265rzB.csv
Reading Data Files......
计算三分类30%和70%的阈值分割点
Now Processing 2013-1 date.......


-v 显示详细信息，包括参数列表parameters

|参数|说明|
|----|----|
|pre|预测资料：../tmp/data.nc|
|type|保存类型：single，保存到单一文件，文件名即下面的name：tmp265rzB.csv|
|obs|实际观测资料：../tmp/obpre.nc|
|name|文件名|

> 这里要计算的时间比较长，所以跑完第一个2013年1月的数据就暂停了！

### 2.1.2 示例：多文件保存

In [1]:
%run ../src/Serialize.py ../tmp/data.nc ../tmp/obpre.nc -v --type multiple --name ./tmp/CSM_PREC

============parameters===========
pre ../tmp/data.nc
type multiple
obs ../tmp/obpre.nc
name ./tmp/CSM_PREC
Reading Data Files......
计算三分类30%和70%的阈值分割点
Now Processing 2013-1 date.......
Complete 0.0 %
Saving data to ./tmp/CSM_PREC_2013-1.csv


IOError: [Errno 2] No such file or directory: u'./tmp/CSM_PREC_2013-1.csv'

在使用多文件保存的时候要注意几个地方，除了命令行中设置--type multiple外，一般都另外设置保存路径和名称，如上例所示我们指定了保存路径为当前路径下的tmp目录，文件名设置为CSM_PREC,表示后面所有的命名都是这样开头的，例如处理到2013年1月的数据，则最终的保存路径就由2部分决定：url=./tmp/CSM_PREC_2013-1.csv,因此我们一般在后面不加.csv的后缀，--name变成了文件url的前缀prefix。

> 注意在notebook中运行时报错，是io错误，可能因为权限问题导致无法创建文件，用户在实际命令行操作时不会出现这种情况，下面是两张运行的截图。

![multiple 1](../tmp/multiple_2.png)
> 上图显示部分文件的处理，下图是生成的csv文件。设计文件保存格式是为了防止大文件的产生和方便用户单独测试需要的时间。

![multiple 2](../tmp/multiple_1.png)

## 3.绘图
制图总共分为两类，标准的ROC曲线和基于gis的网格图。如下图所示：
![roc](http://scikit-learn.org/stable/_images/plot_roc_001.png)
![grid](../tmp/gridROC.jpg)
在src的目录中，ROC.py和BSS.py分别负责两类统计的制图工作，首先来看roc（运行－h查看帮助信息）

### 3.1.1 ROC显示帮助信息

In [1]:
%run ../src/ROC.py -h

usage: ROC.py [-h] [-n NAME] [-t TITLE] [-v] ex index

positional arguments:
  ex                    指定一个序列化后的样本文件
  index                 输出三分类中的哪一层:0:below; 1:normal; 2:above

optional arguments:
  -h, --help            show this help message and exit
  -n NAME, --name NAME  输出结果文件名
  -t TITLE, --title TITLE
                        plot image title
  -v, --verbose         输出详细信息


必填的有2个，ex（example）表示上一步序列化后的样本数据，index指定三分类中的哪一个进行绘图。
- 0:below normal
- 1:normal
- 2:above normal

可选的参数有：
- -n,--name:图件的名称，前面可以指定路径，但必须是绝对路径，例如‘～’等命令行alias暂时不支持,默认随机生成一个文件名，图片后缀png
- -t,--titile:图片上方的主标题，默认是ROC的英文单词
- -v,--verbose:输出详细的过程信息,包括获取的参数列表和ROC曲线上的坐标数据

### 3.1.2 示例：规定样本文件test.csv 和index=0

In [27]:
%run ../src/ROC.py ../tmp/test.csv 0

saving image to ./tmpq5OZA3.png
Completely Finshed.


指定样本文件路径为：../tmp/test.csv ，绘制below normal的ROC图件。因为没有指定文件名和title，因此默认保存在当前目录中，文件名为tmpq5OZA3.png,如下图所示：
![](../tmp/tmpq5OZA3.png)

### 3.1.3 示例：指定图件名和保存路径

In [1]:
%run ../src/ROC.py ../tmp/test.csv 2 --name=../tmp/ROC_Above_normal.png

saving image to ../tmp/ROC_Above_normal.png
Completely Finshed.


指定样本文件路径为：../tmp/test.csv ，绘制above normal的ROC图件。
--name指定保存路径为../tmp,文件名为ROC_Above_normal.png。如下图所示：
![](../tmp/ROC_Above_normal.png)

### 3.1.4 示例：指定图件title

In [2]:
%run ../src/ROC.py ../tmp/test.csv 2 --title='CSM1990-2000 Monthly ROC Curve' --name=../tmp/csm_title.png

saving image to ../tmp/csm_title.png
Completely Finshed.


指定样本文件路径为：../tmp/testcsv ，绘制above normal的ROC图件。
--name指定保存路径为../tmp,文件名为csm_title..png。如下图所示：
![](../tmp/csm_title.png)

### 3.1.5 示例：显示详细信息

In [2]:
%run ../src/ROC.py ../tmp/test.csv 2 --name=../tmp/roc_verbose.png --verbose

============parameters===========
index 2
ex ../tmp/test.csv
name ../tmp/roc_verbose.png
====False Positive Ratio(fpr) And True Positive Ratio(tpr) Pair====
0 0.0 0.0
1 0.000126552909662 0.000126552909662
2 0.000210921516104 0.000210921516104
3 0.000696041003143 0.000696041003143
4 0.00189829364493 0.00189829364493
5 0.00423952247369 0.00423952247369
6 0.00892198013119 0.00892198013119
7 0.0180970660817 0.0180970660817
8 0.0355402754635 0.0355402754635
9 0.0613148847314 0.0613148847314
10 0.102318027462 0.102318027462
11 0.162515028158 0.162515028158
12 0.237349982072 0.237349982072
13 0.326928349961 0.326928349961
14 0.428149585539 0.428149585539
15 0.53293539474 0.53293539474
16 0.634325367531 0.634325367531
17 0.726687899433 0.726687899433
18 0.797220054418 0.797220054418
19 0.855181287043 0.855181287043
20 0.898968593786 0.898968593786
21 0.940119381578 0.940119381578
22 1.0 1.0
saving image to ../tmp/roc_verbose.png
Completely Finshed.


除了最后两行相同外，verbose选项另外显示了配置的参数信息（parameters）和ROC曲线上的坐标点，分三列，从左往右分别是编号，假阳率和真阳率数值。